In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_control = pd.read_csv('data_sensor_01.csv')
df_exp = pd.read_csv('data_sensor_02.csv')

In [3]:
df_control.describe()

,Temperature (°C),Humidity (%),Pressure (kPa),Gas Resistance (KOhm)
count,31599.000000,31599.000000,31599.000000,31599.000000
mean,19.362706,65.757044,100.906309,151.508433
std,1.266416,3.148382,0.996180,12.209348
min,18.370000,49.990000,74.380000,0.000000
25%,18.800000,64.520000,100.860000,149.130000
50%,19.120000,66.110000,100.940000,152.960000
75%,19.680000,67.480000,101.040000,156.690000
max,33.030000,100.000000,101.130000,177.900000


In [4]:
df_exp.describe()

,Timestamp,Temperature (°C),Humidity (%),Pressure (kPa),Gas Resistance (KOhm)
count,6496,6496,6496,6496,6496
unique,6496,413,1345,94,1147
top,2025-11-14 18:21:41,18.76,50.28,100.83,36.28
freq,1,91,97,220,43


In [5]:
df_control = df_control[::5]
df_control.describe()

,Temperature (°C),Humidity (%),Pressure (kPa),Gas Resistance (KOhm)
count,6320.000000,6320.000000,6320.000000,6320.000000
mean,19.362345,65.750693,100.909728,151.511370
std,1.261369,3.129675,0.950456,12.149382
min,18.380000,50.000000,74.380000,0.000000
25%,18.800000,64.510000,100.860000,149.130000
50%,19.120000,66.110000,100.940000,152.960000
75%,19.680000,67.470000,101.040000,156.690000
max,33.030000,100.000000,101.130000,176.030000


In [9]:
# Independent two-sample t-test on 'gas resistance' between control and experimental groups
import scipy.stats as stats
# Extract numeric series and drop missing values
control = pd.to_numeric(df_control['Gas Resistance (KOhm)'], errors='coerce').dropna()
exp = pd.to_numeric(df_exp['Gas Resistance (KOhm)'], errors='coerce').dropna()
# Basic summary
n1 = control.size
n2 = exp.size
m1 = control.mean()
m2 = exp.mean()
s1 = control.std(ddof=1)
s2 = exp.std(ddof=1)
print(f"Control: n={n1}, mean={m1:.4f}, sd={s1:.4f}")
print(f"Experiment: n={n2}, mean={m2:.4f}, sd={s2:.4f}")
# Levene's test for equal variances
lev_stat, lev_p = stats.levene(control, exp)
equal_var = lev_p > 0.05
print(f"Levene test: stat={lev_stat:.4f}, p={lev_p:.4f} -> equal_var={equal_var}")
# Two-sample t-test (uses equal_var depending on Levene)
t_stat, p_val = stats.ttest_ind(control, exp, equal_var=equal_var)
print(f"t-statistic={t_stat:.4f}, p-value={p_val:.4f}")
# 95% CI for the difference in means (Welch approximation)
diff = m1 - m2
se = (s1**2 / n1 + s2**2 / n2) ** 0.5
# Welch-Satterthwaite degrees of freedom
num = (s1**2 / n1 + s2**2 / n2)**2
den = (s1**4) / (n1**2 * (n1 - 1)) if n1 > 1 else 0
den += (s2**4) / (n2**2 * (n2 - 1)) if n2 > 1 else 0
df_welch = num / den if den != 0 else min(n1, n2) - 1
from math import isnan
if se == 0 or isnan(se):
    print("Cannot compute CI (zero or invalid standard error).")
else:
    t_crit = stats.t.ppf(1 - 0.025, df=df_welch)
    ci_low = diff - t_crit * se
    ci_high = diff + t_crit * se
    print(f"Difference in means (control - experiment) = {diff:.4f}")
    print(f"95% CI = [{ci_low:.4f}, {ci_high:.4f}] (df~{df_welch:.2f})")

Control: n=6320, mean=151.5114, sd=12.1494
Experiment: n=6495, mean=48.4817, sd=21.1052
Levene test: stat=2823.3689, p=0.0000 -> equal_var=False
t-statistic=339.7970, p-value=0.0000
Difference in means (control - experiment) = 103.0297
95% CI = [102.4353, 103.6240] (df~10427.47)
